# Testing Notebook

## 1. Some Theory about Recommender Systems

The main families of methods for RecSys are:

- Collaborative Filtering: This method makes automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on a set of items, A is more likely to have B's opinion for a given item than that of a randomly chosen person.

- Content-Based Filtering: This method uses only information about the description and attributes of the items users has previously consumed to model user's preferences. In other words, these algorithms try to recommend items that are similar to those that a user liked in the past (or is examining in the present). In particular, various candidate items are compared with items previously rated by the user and the best-matching items are recommended.

- Hybrid methods: Recent research has demonstrated that a hybrid approach, combining collaborative filtering and content-based filtering could be more effective than pure approaches in some cases. These methods can also be used to overcome some of the common problems in recommender systems such as cold start and the sparsity problem.

https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

In [2]:
import scipy
import math
import random
import sklearn
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler

Example: https://www.kaggle.com/code/hendraherviawan/itembased-collaborative-filter-recommendation-r/report

## 2 Read Data from Parquets

In [3]:
transactions = pd.read_parquet('data/transactions_train_sample_gt15transactions.parquet')
customers = pd.read_parquet('data/customers_train_sample_gt15transactions.parquet')
articles = pd.read_parquet('data/articles_train_sample_gt15transactions.parquet')

## 3. Join Dataframes

In [4]:
#### Join Transactions and Articles dataframes
transactions_articles_joined = transactions.join(articles.set_index('article_id'), on='article_id')

In [5]:
transactions_articles_joined.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,1728846800780188,519773001,0.028458,2,55,519773,7147,245,17,...,1,0,0,1,0,15,0,1003,3,10231
1,2018-09-20,1728846800780188,578472001,0.032525,2,55,578472,37340,263,38,...,23,0,0,1,0,19,40,1007,9,26053
2,2018-09-20,2076973761519164,661795002,0.167797,2,55,661795,43993,263,38,...,23,0,0,1,0,19,40,1007,9,32892
3,2018-09-20,2076973761519164,684080003,0.101678,2,55,684080,1768,262,6,...,23,0,0,1,0,19,40,1007,9,6151
4,2018-09-20,65343934020628982,534357005,0.036593,2,55,534357,10920,272,0,...,60,1,1,2,2,57,37,1016,11,305


In [6]:
#### Join Transactions-Articles with Customers dataframes
features_joined = transactions_articles_joined.join(customers.set_index('customer_id'), on='customer_id')

In [7]:
features_joined.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,club_member_status,fashion_news_frequency,age,postal_code
0,2018-09-20,1728846800780188,519773001,0.028458,2,55,519773,7147,245,17,...,0,15,0,1003,3,10231,0,0,59.0,44730
1,2018-09-20,1728846800780188,578472001,0.032525,2,55,578472,37340,263,38,...,0,19,40,1007,9,26053,0,0,59.0,44730
2,2018-09-20,2076973761519164,661795002,0.167797,2,55,661795,43993,263,38,...,0,19,40,1007,9,32892,0,0,55.0,18589
3,2018-09-20,2076973761519164,684080003,0.101678,2,55,684080,1768,262,6,...,0,19,40,1007,9,6151,0,0,55.0,18589
4,2018-09-20,65343934020628982,534357005,0.036593,2,55,534357,10920,272,0,...,2,57,37,1016,11,305,0,0,24.0,115923


In [8]:
len(features_joined)

585296

## 4 Drop unused columns

Collaborative Filtering only needs customers_id and articles_id

In [9]:
features_prepared = features_joined[['customer_id', 'article_id']]

## 5 Collaborative filtering

Build a matrix of items for the client

### 5.1 Build customer-articles matrix

In [10]:
#Add column quantity
features_prepared['Quantity'] = 1

/tmp/ipykernel_1556119/439893453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_prepared['Quantity'] = 1


In [11]:
features_prepared.head()

,customer_id,article_id,Quantity
0,1728846800780188,519773001,1
1,1728846800780188,578472001,1
2,2076973761519164,661795002,1
3,2076973761519164,684080003,1
4,65343934020628982,534357005,1


In [12]:
customer_item_matrix = features_prepared.pivot_table(
    index='customer_id', 
    columns='article_id', 
    values='Quantity',
    aggfunc='sum'
)

In [13]:
customer_item_matrix

article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,...,828718002,829135001,829783001,829783005,829971001,829971002,830755001,832616001,834031001,834292001
customer_id,,,,,,,,,,,,,,,,,,,,,
1728846800780188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1845857727772358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2076973761519164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8421185370332018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8979857957419452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18439734652989632972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18441768191369883939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18442722120177658597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 We now have a matrix where each row represents the total quantities purchased for each product for each customer.

let's code 0-1 this data, so that a value of 1 means that the given product was bought by the given customer, and a value of 0 means that the given product was never bought by the given customer. Take a look at the following code:

In [14]:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)

In [15]:
customer_item_matrix

article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,...,828718002,829135001,829783001,829783005,829971001,829971002,830755001,832616001,834031001,834292001
customer_id,,,,,,,,,,,,,,,,,,,,,
1728846800780188,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1845857727772358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2076973761519164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8421185370332018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8979857957419452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18439734652989632972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18441768191369883939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18442722120177658597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5.2 Build Customers Similarity Matrix

Calculate the cosine similarities between users

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
user_user_sim_matrix = pd.DataFrame(
    cosine_similarity(customer_item_matrix)
)

In [18]:
user_user_sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,14964,14965,14966,14967,14968,14969,14970,14971,14972,14973
0,1.000000,0.0,0.022178,0.0,0.0,0.0,0.000000,0.018270,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,0.000000,1.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,0.022178,0.0,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.032774,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,0.000000,0.0,0.000000,0.0,1.0,0.0,0.029814,0.000000,0.0,0.027217,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14969,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0
14970,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0
14971,0.000000,0.0,0.000000,0.0,0.0,0.0,0.041345,0.030261,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.036736,0.0
14972,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.054074,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.036736,1.000000,0.0


In [19]:
user_user_sim_matrix.columns = customer_item_matrix.index

In [20]:
user_user_sim_matrix['customer_id'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('customer_id')

In [21]:
user_user_sim_matrix

customer_id,1728846800780188,1845857727772358,2076973761519164,8421185370332018,8979857957419452,9020202416008033,9983268471081545,10178278581755647,10838479868465930,12554419226309955,...,18435221511488011015,18435362578049473419,18436849338957934485,18436877197079302071,18437277199235917412,18439734652989632972,18441768191369883939,18442722120177658597,18444595675436699040,18445051157201360796
customer_id,,,,,,,,,,,,,,,,,,,,,
1728846800780188,1.000000,0.0,0.022178,0.0,0.0,0.0,0.000000,0.018270,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1845857727772358,0.000000,1.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2076973761519164,0.022178,0.0,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
8421185370332018,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.032774,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
8979857957419452,0.000000,0.0,0.000000,0.0,1.0,0.0,0.029814,0.000000,0.0,0.027217,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18439734652989632972,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0
18441768191369883939,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0
18442722120177658597,0.000000,0.0,0.000000,0.0,0.0,0.0,0.041345,0.030261,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.036736,0.0


These are the 10 most similar clients to the b14bfba3ae0da5af6e9711059773acf713cd7bb9a2c940ddf570affb715988a0 client. Let's choose client 26f41c2913090e7a620df05975a52c604caf017c3110bb62596fdebd1aae4ba9 and discuss how we can recommend products using these results.

Lets identify both users:
- user_A: b14bfba3ae0da5af6e9711059773acf713cd7bb9a2c940ddf570affb715988a0
- user_B: 26f41c2913090e7a620df05975a52c604caf017c3110bb62596fdebd1aae4ba9

The strategy is as follows.

- First, we need to identify the items that user_A and user2 customers have already purchased.
- Then, let's find the products that target client user_B has not purchased, but client user_A has.
- Since these two customers have bought similar items in the past, we will assume that target customer user_B has a high probability of buying the items he or she has not bought, but customer user_A has.
- Finally, we are going to use this list of items and recommend them to target customer user_B.

### 5.3 Build the recommendations for each customer

In [32]:
# Create empty dataframe that will store the customers and their recommended products
column_names = ["customer_id", "recom_articles"]
customer_rec_articles_df = pd.DataFrame(columns = column_names)

In [33]:
%%capture --no-display 

for i in range(len(customer_item_matrix)-1):   
    customer_i_id = customer_item_matrix.index[i]
    items_to_recommend_to_i = set()

    items_bought_by_i = set(customer_item_matrix.loc[customer_i_id].iloc[customer_item_matrix.loc[customer_i_id].to_numpy().nonzero()].index)
    
    #Get most similar users to current user i
    similar_users = user_user_sim_matrix.iloc[i].sort_values(ascending=False)[0:10] #get just the 10 most similar clients
    similar_users = similar_users[similar_users!=0].to_frame() #Get only clients with similarity score greater than 0

    #Get articles bought by the j similar users
    for j in similar_users.index.values:
        customer_j_id = j
        items_bought_by_j = set(customer_item_matrix.loc[customer_j_id].iloc[customer_item_matrix.loc[customer_j_id].to_numpy().nonzero()].index)
        items_to_recommend = items_bought_by_j - items_bought_by_i
        #if(len(items_to_recommend)>0): items_to_recommend_to_i.append(items_to_recommend)
        items_to_recommend_to_i = set.union(items_to_recommend_to_i, items_to_recommend)

    customer_rec_articles_df = customer_rec_articles_df.append({'customer_id': customer_i_id, 'recom_articles':items_to_recommend_to_i}, ignore_index=True)


In [34]:
#Convert set of recommended articles to list
customer_rec_articles_df['recom_articles'] = customer_rec_articles_df['recom_articles'].apply(lambda x: list(x))

In [35]:
len(customer_rec_articles_df)

14973

In [36]:
customer_rec_articles_df.head()

,customer_id,recom_articles
0,1728846800780188,"[748032001, 600043010, 712320003, 712320002, 6..."
1,1845857727772358,"[708352001, 490176002, 600043011, 557248001, 6..."
2,2076973761519164,"[752512003, 716672007, 687061002, 212629004, 7..."
3,8421185370332018,"[648256001, 599083009, 678571011, 341782018, 6..."
4,8979857957419452,"[573856001, 735648002, 706016003, 708459011, 7..."


Export dataframe to csv:

In [37]:
customer_rec_articles_df.to_csv('customers_recommended_articles.csv', index=False)

## 6 FUTURE: Content-Based Filtering

https://www.kaggle.com/code/fabiendaniel/film-recommendation-engine/notebook

SIMILARITY

Criteria to determine if two products are similar:
1. Section and Type (both together)
2. Graphical appearence
3. Colour